In [156]:
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException
import numpy as np
import pandas as pd
import random
import re
from sklearn.model_selection import train_test_split

In [54]:
def strip_emoticons(s):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', s.strip())

In [132]:
def add_cleaned_col(df):
    index = 0
    clean_text = {}
    for tt_text in df.text:
        if type(df.retweeted_status[index]) is not float:
            if tt_text[:2] == 'RT':
                tt_text = tt_text[2:]
        tt_text = strip_emoticons(tt_text)
        tt_text = re.sub('@[a-zA-Z0-9_.:-]+','',tt_text) # retira '@'
        tt_text = re.sub('(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]|(\xa0(…)*)))+){1}','',tt_text) # retira url
        tt_text = re.sub('\n',' ',tt_text) # retira '\n'
        clean_text[index] = tt_text
        index +=1
    df['cleaned_text'] = pd.Series(clean_text)
    return df

### Pre-processando os textos dos tweets

In [155]:
#dados_chunk = pd.read_json("100k_origin.json",convert_dates=False,lines=True,chunksize=15000)
#for df in dados_chunk:
#    dados = df
#    break

In [148]:
dados = pd.read_json("5k_origin.json",convert_dates=False,lines=True)
dados.head()

,_id,quote_count,quoted_status_permalink,contributors,truncated,text,is_quote_status,in_reply_to_status_id,reply_count,id,...,created_at,quoted_status_id_str,filter_level,in_reply_to_status_id_str,place,control,possibly_sensitive,extended_entities,extended_tweet,display_text_range
0,{'$numberLong': '1240986587703529472'},0,"{'url': 'https://t.co/erjndGmPS3', 'expanded':...",NaN,False,RT @collyfirth: herdeiro de quê meu deus,True,None,0,{'$numberLong': '1240986587703529472'},...,{'$date': '2020-03-20T10:00:33.000-0300'},1.240647e+18,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN
1,{'$numberLong': '1240986588387004417'},0,NaN,NaN,False,O Jair Banana @jairbolsonaro foi pros Estados ...,False,None,0,{'$numberLong': '1240986588387004417'},...,{'$date': '2020-03-20T10:00:33.000-0300'},NaN,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN
2,{'$numberLong': '1240986588810805250'},0,NaN,NaN,False,"RT @Negro_Brahma: Dia 3\n\nCOVID-19, tal do co...",False,None,0,{'$numberLong': '1240986588810805250'},...,{'$date': '2020-03-20T10:00:33.000-0300'},NaN,low,NaN,None,{'coleta': [439]},0.0,{'media': [{'additional_media_info': {'monetiz...,NaN,NaN
3,{'$numberLong': '1240986588819132417'},0,NaN,NaN,False,RT @xxxwelzinho: O corona quando a vacina for ...,False,None,0,{'$numberLong': '1240986588819132417'},...,{'$date': '2020-03-20T10:00:33.000-0300'},NaN,low,NaN,None,{'coleta': [439]},0.0,{'media': [{'additional_media_info': {'monetiz...,NaN,NaN
4,{'$numberLong': '1240986589150527495'},0,NaN,NaN,False,RT @gelleejeanluc: corona level 4 😷💀😷💀😂😂😂😂 htt...,False,None,0,{'$numberLong': '1240986589150527495'},...,{'$date': '2020-03-20T10:00:33.000-0300'},NaN,low,NaN,None,{'coleta': [439]},1.0,{'media': [{'additional_media_info': {'monetiz...,NaN,NaN


In [149]:
dados

,_id,quote_count,quoted_status_permalink,contributors,truncated,text,is_quote_status,in_reply_to_status_id,reply_count,id,...,created_at,quoted_status_id_str,filter_level,in_reply_to_status_id_str,place,control,possibly_sensitive,extended_entities,extended_tweet,display_text_range
0,{'$numberLong': '1240986587703529472'},0,"{'url': 'https://t.co/erjndGmPS3', 'expanded':...",NaN,False,RT @collyfirth: herdeiro de quê meu deus,True,None,0,{'$numberLong': '1240986587703529472'},...,{'$date': '2020-03-20T10:00:33.000-0300'},1.240647e+18,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN
1,{'$numberLong': '1240986588387004417'},0,NaN,NaN,False,O Jair Banana @jairbolsonaro foi pros Estados ...,False,None,0,{'$numberLong': '1240986588387004417'},...,{'$date': '2020-03-20T10:00:33.000-0300'},NaN,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN
2,{'$numberLong': '1240986588810805250'},0,NaN,NaN,False,"RT @Negro_Brahma: Dia 3\n\nCOVID-19, tal do co...",False,None,0,{'$numberLong': '1240986588810805250'},...,{'$date': '2020-03-20T10:00:33.000-0300'},NaN,low,NaN,None,{'coleta': [439]},0.0,{'media': [{'additional_media_info': {'monetiz...,NaN,NaN
3,{'$numberLong': '1240986588819132417'},0,NaN,NaN,False,RT @xxxwelzinho: O corona quando a vacina for ...,False,None,0,{'$numberLong': '1240986588819132417'},...,{'$date': '2020-03-20T10:00:33.000-0300'},NaN,low,NaN,None,{'coleta': [439]},0.0,{'media': [{'additional_media_info': {'monetiz...,NaN,NaN
4,{'$numberLong': '1240986589150527495'},0,NaN,NaN,False,RT @gelleejeanluc: corona level 4 😷💀😷💀😂😂😂😂 htt...,False,None,0,{'$numberLong': '1240986589150527495'},...,{'$date': '2020-03-20T10:00:33.000-0300'},NaN,low,NaN,None,{'coleta': [439]},1.0,{'media': [{'additional_media_info': {'monetiz...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,{'$numberLong': '1240987852923473922'},0,"{'url': 'https://t.co/w1fDLPAlIH', 'expanded':...",NaN,False,RT @pullupnghost: Imaginem q as pessoas depois...,True,None,0,{'$numberLong': '1240987852923473922'},...,{'$date': '2020-03-20T10:05:35.000-0300'},1.239823e+18,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN
4996,{'$numberLong': '1240987853015977984'},0,"{'url': 'https://t.co/vkes4QcyPL', 'expanded':...",NaN,False,alguém prende esse imbecil do caralho,True,None,0,{'$numberLong': '1240987853015977984'},...,{'$date': '2020-03-20T10:05:35.000-0300'},1.240985e+18,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN
4997,{'$numberLong': '1240987853305319424'},0,NaN,NaN,True,“É impressionante como a mídia televisiva não ...,False,None,0,{'$numberLong': '1240987853305319424'},...,{'$date': '2020-03-20T10:05:35.000-0300'},NaN,low,NaN,None,{'coleta': [439]},0.0,NaN,"{'display_text_range': [0, 195], 'entities': {...",NaN
4998,{'$numberLong': '1240987853380882432'},0,NaN,NaN,False,Exclusivo: Busca por trabalhadores autônomos c...,False,None,0,{'$numberLong': '1240987853380882432'},...,{'$date': '2020-03-20T10:05:35.000-0300'},NaN,low,NaN,None,{'coleta': [439]},0.0,NaN,NaN,NaN


In [150]:
dados_cld = add_cleaned_col(dados)
dados_cld.head()

,_id,quote_count,quoted_status_permalink,contributors,truncated,text,is_quote_status,in_reply_to_status_id,reply_count,id,...,quoted_status_id_str,filter_level,in_reply_to_status_id_str,place,control,possibly_sensitive,extended_entities,extended_tweet,display_text_range,cleaned_text
0,{'$numberLong': '1240986587703529472'},0,"{'url': 'https://t.co/erjndGmPS3', 'expanded':...",NaN,False,RT @collyfirth: herdeiro de quê meu deus,True,None,0,{'$numberLong': '1240986587703529472'},...,1.240647e+18,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN,herdeiro de quê meu deus
1,{'$numberLong': '1240986588387004417'},0,NaN,NaN,False,O Jair Banana @jairbolsonaro foi pros Estados ...,False,None,0,{'$numberLong': '1240986588387004417'},...,NaN,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN,O Jair Banana foi pros Estados Unidos com nos...
2,{'$numberLong': '1240986588810805250'},0,NaN,NaN,False,"RT @Negro_Brahma: Dia 3\n\nCOVID-19, tal do co...",False,None,0,{'$numberLong': '1240986588810805250'},...,NaN,low,NaN,None,{'coleta': [439]},0.0,{'media': [{'additional_media_info': {'monetiz...,NaN,NaN,"Dia 3 COVID-19, tal do coronavirus, querendo..."
3,{'$numberLong': '1240986588819132417'},0,NaN,NaN,False,RT @xxxwelzinho: O corona quando a vacina for ...,False,None,0,{'$numberLong': '1240986588819132417'},...,NaN,low,NaN,None,{'coleta': [439]},0.0,{'media': [{'additional_media_info': {'monetiz...,NaN,NaN,O corona quando a vacina for criada
4,{'$numberLong': '1240986589150527495'},0,NaN,NaN,False,RT @gelleejeanluc: corona level 4 😷💀😷💀😂😂😂😂 htt...,False,None,0,{'$numberLong': '1240986589150527495'},...,NaN,low,NaN,None,{'coleta': [439]},1.0,{'media': [{'additional_media_info': {'monetiz...,NaN,NaN,corona level 4


### Dividindo dados para treino e teste

In [153]:
random.seed(1000)

data_api, data_test = train_test_split(dados_cld, train_size=2500, random_state=0)

In [154]:
data_test

,_id,quote_count,quoted_status_permalink,contributors,truncated,text,is_quote_status,in_reply_to_status_id,reply_count,id,...,quoted_status_id_str,filter_level,in_reply_to_status_id_str,place,control,possibly_sensitive,extended_entities,extended_tweet,display_text_range,cleaned_text
398,{'$numberLong': '1240986690711441408'},0,NaN,NaN,False,RT @rolealeatorio: daniel no paredão \ncom o b...,False,None,0,{'$numberLong': '1240986690711441408'},...,NaN,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN,daniel no paredão com o brasil inteiro em ca...
3833,{'$numberLong': '1240987559163056131'},0,NaN,NaN,False,RT @gabieesteves: Em Moçambique ainda não temo...,False,None,0,{'$numberLong': '1240987559163056131'},...,NaN,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN,Em Moçambique ainda não temos nenhum Caso con...
4836,{'$numberLong': '1240987808212410374'},0,NaN,NaN,False,RT @harryesirr: se o corona nao me matar o har...,False,None,0,{'$numberLong': '1240987808212410374'},...,NaN,low,NaN,None,{'coleta': [439]},0.0,{'media': [{'source_user_id': {'$numberLong': ...,NaN,NaN,se o corona nao me matar o harry me mata é isto
4572,{'$numberLong': '1240987736481333249'},0,"{'url': 'https://t.co/erjndGmPS3', 'expanded':...",NaN,False,RT @collyfirth: herdeiro de quê meu deus,True,None,0,{'$numberLong': '1240987736481333249'},...,1.240647e+18,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN,herdeiro de quê meu deus
636,{'$numberLong': '1240986750241189889'},0,NaN,NaN,False,"Primeiro dia de Quarentena, era melhor ter peg...",False,None,0,{'$numberLong': '1240986750241189889'},...,NaN,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN,"Primeiro dia de Quarentena, era melhor ter peg..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,{'$numberLong': '1240986911696658432'},0,NaN,NaN,False,RT @rolealeatorio: daniel no paredão \ncom o b...,False,None,0,{'$numberLong': '1240986911696658432'},...,NaN,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN,daniel no paredão com o brasil inteiro em ca...
4794,{'$numberLong': '1240987795268743168'},0,NaN,NaN,False,Todos a pensar no Coronavirus e eu a pensar em...,False,None,0,{'$numberLong': '1240987795268743168'},...,NaN,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN,Todos a pensar no Coronavirus e eu a pensar em...
3869,{'$numberLong': '1240987566465302530'},0,NaN,NaN,False,RT @rolealeatorio: daniel no paredão \ncom o b...,False,None,0,{'$numberLong': '1240987566465302530'},...,NaN,low,NaN,None,{'coleta': [439]},NaN,NaN,NaN,NaN,daniel no paredão com o brasil inteiro em ca...
2651,{'$numberLong': '1240987263166709761'},0,"{'url': 'https://t.co/upKztc5DOg', 'expanded':...",NaN,True,APOIO que se REPASSE à SAÚDE tanto o dinheiro ...,True,None,0,{'$numberLong': '1240987263166709761'},...,1.240799e+18,low,NaN,None,{'coleta': [439]},NaN,NaN,"{'display_text_range': [0, 124], 'entities': {...",NaN,APOIO que se REPASSE à SAÚDE tanto o dinheiro ...


In [165]:
authenticator = IAMAuthenticator('')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url('')
tone_analyzer.set_disable_ssl_verification(False)

In [164]:
tones = []
index = 0
for item in data_test['cleaned_text']:
    index = index + 1
    text = item
    tone_analysis = tone_analyzer.tone({'text': text}, content_type='application/json').get_result()
    tones.append(tone_analysis)

ERROR:root:<HTML><HEAD>
<TITLE>Service Unavailable</TITLE>
</HEAD><BODY>
<H1>Service Unavailable - Zero size object</H1>
The server is temporarily unable to service your request.  Please try again
later.<P>
Reference&#32;&#35;15&#46;8ca3db17&#46;1595204523&#46;456e896f
</BODY></HTML>
Traceback (most recent call last):
  File "C:\Users\vicda\Anaconda3\lib\site-packages\ibm_cloud_sdk_core\base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: <HTML><HEAD>
<TITLE>Service Unavailable</TITLE>
</HEAD><BODY>
<H1>Service Unavailable - Zero size object</H1>
The server is temporarily unable to service your request.  Please try again
later.<P>
Reference&#32;&#35;15&#46;8ca3db17&#46;1595204523&#46;456e896f
</BODY></HTML>
, Code: 503


ApiException: Error: <HTML><HEAD>
<TITLE>Service Unavailable</TITLE>
</HEAD><BODY>
<H1>Service Unavailable - Zero size object</H1>
The server is temporarily unable to service your request.  Please try again
later.<P>
Reference&#32;&#35;15&#46;8ca3db17&#46;1595204523&#46;456e896f
</BODY></HTML>
, Code: 503

In [ ]:
tones